In [10]:
import pandas as pd
from openai import OpenAI
import random
import os
import re

In [11]:
category = "gerçek"
idiom = "Zaman kazanmak"
word1 = "Zaman"
word2 = "kazanmak"
model ="gpt-4-turbo"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="OpenAI Key",
)

In [12]:
if not os.path.exists('response_data.csv'):
    pd.DataFrame(columns=["raw_response","category", "idiom", "type"]).to_csv('response_data.csv', index=False)


In [13]:
df_structured = pd.DataFrame(columns=["submission", "category", "idiom", "type"])

In [14]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Türkçe, anadiliniz olduğundan, dilimizi zengin ve yaratıcı biçimlerde kullanmalısın. Lütfen cümlelerinizde benzer yapılar kullanmaktan kaçının.İnsan isimlerini kullanarak cümle kurmamalısın ! İstemde belirtilen kelimeleri gerçek, somut veya fiziki anlama gelecek bir şekilde kullanmalısın.İstemde belirtilen kelimeleri mecaz anlamda kullanmamalısın !"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [15]:
chat_session = ChatSession(category=category)

Türkçe, anadiliniz olduğundan, dilimizi zengin ve yaratıcı biçimlerde kullanmalısın. Lütfen cümlelerinizde benzer yapılar kullanmaktan kaçının.İnsan isimlerini kullanarak cümle kurmamalısın ! İstemde belirtilen kelimeleri gerçek, somut veya fiziki anlama gelecek bir şekilde kullanmalısın.İstemde belirtilen kelimeleri mecaz anlamda kullanmamalısın !


In [16]:
type = "Zero-shot"
step = 1

In [17]:
prompt = f"""
"{word1}" ve "{word2}" bu iki kelimenin fiziksel bir eylemi ifade edecek şekilde kullanılarak geçtiği 5 örnek cümle yaz. İstemde belirtilen kelimeleri köklerini değiştirmeden istediğin gibi çekimleyebilirsin. İstemde belirtilen kelimeleri cümlede istediğin yerlere koyabilirsin ama iki kelimenin de geçtiği cümle kurmalısın. Yanıt olarak sadece cümleleri dön.
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


"Zaman" ve "kazanmak" bu iki kelimenin fiziksel bir eylemi ifade edecek şekilde kullanılarak geçtiği 5 örnek cümle yaz. İstemde belirtilen kelimeleri köklerini değiştirmeden istediğin gibi çekimleyebilirsin. İstemde belirtilen kelimeleri cümlede istediğin yerlere koyabilirsin ama iki kelimenin de geçtiği cümle kurmalısın. Yanıt olarak sadece cümleleri dön.





In [18]:
responses

['1. Atletler, yarışı kazanmak için zamanlarını iyi ayarlamalı.\n2. Şef, yemeği zamanında hazırlayıp müşterileri kazanmak zorundaydı.\n3. Eğitmen, sınava yetiştirebilmek için doğru zamanlamayla ders planını kazanmak istedi.\n4. Çiftçi, hasat zamanını doğru belirleyerek pazarda rekabet avantajı kazanmak istedi.\n5. Satıcı, indirim zamanlarını iyi belirleyerek daha fazla müşteri kazanmak için stratejisini geliştirdi.']

In [704]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    # Initialize the list to store structured data
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual sentences
        sentences = block.split('\n')
        # Remove the numbering from each sentence and strip leading/trailing whitespace
        clean_sentences = [sentence.split('. ', 1)[-1].strip() for sentence in sentences]
        # For each sentence, append a tuple with the structured data
        for sentence in clean_sentences:
            structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [705]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,Çocuklar birdenbire oyun kutusunun içinden neş...,gerçek,Ortaya çıkmak,Zero-shot
1,"Dağ yürüyüşü sırasında, bir tilki aniden çalıl...",gerçek,Ortaya çıkmak,Zero-shot
2,"Piknik esnasında, toprağın altından bir yılan ...",gerçek,Ortaya çıkmak,Zero-shot
3,"Bahçede çalışırken, eski bir Roma sikkesi bird...",gerçek,Ortaya çıkmak,Zero-shot
4,Bodrum katını temizlerken önemli bir aile yadi...,gerçek,Ortaya çıkmak,Zero-shot
5,Kutunun içinden bir el feneri ortaya çıktı.,gerçek,Ortaya çıkmak,Zero-shot
6,Karanlık odadan aniden bir kediyi ortaya çıktı.,gerçek,Ortaya çıkmak,Zero-shot
7,"Sofra kurulurken, masanın altından bir top ort...",gerçek,Ortaya çıkmak,Zero-shot
8,Çalışma odasındaki eski sandalyenin altından b...,gerçek,Ortaya çıkmak,Zero-shot
9,"Kazı yaparken, bilim insanları fosil bir kabuğ...",gerçek,Ortaya çıkmak,Zero-shot


In [706]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,Çocuklar birdenbire oyun kutusunun içinden neş...,gerçek,Ortaya çıkmak,Zero-shot
1,"Dağ yürüyüşü sırasında, bir tilki aniden çalıl...",gerçek,Ortaya çıkmak,Zero-shot
2,"Piknik esnasında, toprağın altından bir yılan ...",gerçek,Ortaya çıkmak,Zero-shot
3,"Bahçede çalışırken, eski bir Roma sikkesi bird...",gerçek,Ortaya çıkmak,Zero-shot
4,Bodrum katını temizlerken önemli bir aile yadi...,gerçek,Ortaya çıkmak,Zero-shot
5,Kutunun içinden bir el feneri ortaya çıktı.,gerçek,Ortaya çıkmak,Zero-shot
6,Karanlık odadan aniden bir kediyi ortaya çıktı.,gerçek,Ortaya çıkmak,Zero-shot
7,"Sofra kurulurken, masanın altından bir top ort...",gerçek,Ortaya çıkmak,Zero-shot
8,Çalışma odasındaki eski sandalyenin altından b...,gerçek,Ortaya çıkmak,Zero-shot
9,"Kazı yaparken, bilim insanları fosil bir kabuğ...",gerçek,Ortaya çıkmak,Zero-shot


In [707]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,Sabahleyin bahçeden çıkmak için kapıyı araladı...,gerçek,Ortaya çıkmak,Zero-shot
1,Kutuyu açtığında içinden eski bir saat çıkarak...,gerçek,Ortaya çıkmak,Zero-shot
2,Çadırdan tehlikeli bir ses geldiğini duyunca i...,gerçek,Ortaya çıkmak,Zero-shot
3,"Köpek, koltuğun altına gizlenmiş topunu ortaya...",gerçek,Ortaya çıkmak,Zero-shot
4,"Çocuklar sandığı köşeden çıkarıp açınca, içind...",gerçek,Ortaya çıkmak,Zero-shot
5,Küçük kardeşim kutunun içinden gülerek ortaya ...,gerçek,Ortaya çıkmak,Zero-shot
6,Tavşan birden bire çalılıkların arasından orta...,gerçek,Ortaya çıkmak,Zero-shot
7,"Arkeologlar, kazı sırasında antik bir heykelin...",gerçek,Ortaya çıkmak,Zero-shot
8,Denizin altında saklı hazinenin ortaya çıkması...,gerçek,Ortaya çıkmak,Zero-shot
9,Köşedeki karanlık sokaktan bir kediyi takip ed...,gerçek,Ortaya çıkmak,Zero-shot


In [708]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"日本語が母語であるため、言語を豊かで創造的な方法で使うべきです。同じような構造の文を繰り返さないようにしてください。人名を使って文を作成しないでください！リクエストで指定された言葉を具体的、実際的、物理的な意味で使用する必要があります。リクエストで指定された言葉を比喩的な意味で使用しないでください！"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [709]:
chat_session = ChatSession(category=category)

Türkçe, anadiliniz olduğundan, dilimizi zengin ve yaratıcı biçimlerde kullanmalısın. Lütfen cümlelerinizde benzer yapılar kullanmaktan kaçının.İnsan isimlerini kullanarak cümle kurmamalısın ! İstemde belirtilen kelimeleri gerçek, somut veya fiziki anlama gelecek bir şekilde kullanmalısın.İstemde belirtilen kelimeleri mecaz anlamda kullanmamalısın !


In [720]:
type = "Enhanced-prompting"
step = 2

In [721]:
prompt = f"""
Sanki bir insan gibi "{word1}" ve "{word2}" bu iki kelimenin fiziksel bir eylemi ifade edecek şekilde kullanılarak geçtiği 5 örnek cümle yaz.İstemde belirtilen kelimeleri köklerini değiştirmeden istediğin gibi çekimleyebilirsin. İstemde belirtilen kelimeleri cümlede istediğin yerleri koyabilirsin ama iki kelimenin de geçtiği cümle kurmalısın. Lütfen cümlelerinizi tekrar kullanmaktan kaçının. Yanıt olarak sadece cümleleri dön.
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


Sanki bir insan gibi "Ortaya" ve "çıkmak" bu iki kelimenin fiziksel bir eylemi ifade edecek şekilde kullanılarak geçtiği 5 örnek cümle yaz.İstemde belirtilen kelimeleri köklerini değiştirmeden istediğin gibi çekimleyebilirsin. İstemde belirtilen kelimeleri cümlede istediğin yerleri koyabilirsin ama iki kelimenin de geçtiği cümle kurmalısın. Lütfen cümlelerinizi tekrar kullanmaktan kaçının. Yanıt olarak sadece cümleleri dön.





In [722]:
responses

['1. Gecenin bir yarısı, aniden köşeden bir kedi ortaya çıkarak yolumu kesti.\n2. Parkta oynarken, topun peşinden koşan çocuklar birden gözden kaybolup  ardından tekrar ortaya çıktılar.\n3. Denizin ortasında batık bir gemiden dalgıçlar eski bir sandığı ortaya çıkarma çalışması yaptılar.\n4. Karanlık bir sokakta ilerlerken ansızın önüme bir bisikletli ortaya çıktı.\n5. Eski fotoğrafları karıştırırken ninenin gençlik yıllarına ait bir albüm ortaya çıktı.',
 '1. Masanın altından oyuncak arayan çocuk, bir kutu ortaya çıktığında sevinçle zıpladı.\n2. Kütüphane raflarını düzenlerken nadir bir kitap ortaya çıkmakla kalmadı, tarihi anlamda büyük bir keşif oldu.\n3. Bahçede çiçekleri sularken topraktan bir kurbağa ortaya çıktı ve çocuklar büyük bir heyecan yaşadı.\n4. Bilgisayarım bozulduğunda, teknisyen kasayı açınca bir sürü toz ortaya çıktı.\n5. Odanın köşesini temizlerken, yıllardır kayıp olan gümüş yüzük yerde parıldayarak ortaya çıktı.']

In [723]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,"Gecenin bir yarısı, aniden köşeden bir kedi or...",gerçek,Ortaya çıkmak,Enhanced-prompting
1,"Parkta oynarken, topun peşinden koşan çocuklar...",gerçek,Ortaya çıkmak,Enhanced-prompting
2,Denizin ortasında batık bir gemiden dalgıçlar ...,gerçek,Ortaya çıkmak,Enhanced-prompting
3,Karanlık bir sokakta ilerlerken ansızın önüme ...,gerçek,Ortaya çıkmak,Enhanced-prompting
4,Eski fotoğrafları karıştırırken ninenin gençli...,gerçek,Ortaya çıkmak,Enhanced-prompting
5,"Masanın altından oyuncak arayan çocuk, bir kut...",gerçek,Ortaya çıkmak,Enhanced-prompting
6,Kütüphane raflarını düzenlerken nadir bir kita...,gerçek,Ortaya çıkmak,Enhanced-prompting
7,Bahçede çiçekleri sularken topraktan bir kurba...,gerçek,Ortaya çıkmak,Enhanced-prompting
8,"Bilgisayarım bozulduğunda, teknisyen kasayı aç...",gerçek,Ortaya çıkmak,Enhanced-prompting
9,"Odanın köşesini temizlerken, yıllardır kayıp o...",gerçek,Ortaya çıkmak,Enhanced-prompting


In [724]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,Sabahleyin bahçeden çıkmak için kapıyı araladı...,gerçek,Ortaya çıkmak,Zero-shot
1,Kutuyu açtığında içinden eski bir saat çıkarak...,gerçek,Ortaya çıkmak,Zero-shot
2,Çadırdan tehlikeli bir ses geldiğini duyunca i...,gerçek,Ortaya çıkmak,Zero-shot
3,"Köpek, koltuğun altına gizlenmiş topunu ortaya...",gerçek,Ortaya çıkmak,Zero-shot
4,"Çocuklar sandığı köşeden çıkarıp açınca, içind...",gerçek,Ortaya çıkmak,Zero-shot
...,...,...,...,...
75,"Masanın altından oyuncak arayan çocuk, bir kut...",gerçek,Ortaya çıkmak,Enhanced-prompting
76,Kütüphane raflarını düzenlerken nadir bir kita...,gerçek,Ortaya çıkmak,Enhanced-prompting
77,Bahçede çiçekleri sularken topraktan bir kurba...,gerçek,Ortaya çıkmak,Enhanced-prompting
78,"Bilgisayarım bozulduğunda, teknisyen kasayı aç...",gerçek,Ortaya çıkmak,Enhanced-prompting


In [725]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Türkçe, anadiliniz olduğundan, dilimizi zengin ve yaratıcı biçimlerde kullanmalısın. Lütfen cümlelerinizde benzer yapılar kullanmaktan kaçının.İnsan isimlerini kullanarak cümle kurmamalısın ! İstemde belirtilen kelimeleri gerçek, somut veya fiziki anlama gelecek bir şekilde kullanmalısın.İstemde belirtilen kelimeleri mecaz anlamda kullanmamalısın !"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [726]:
chat_session = ChatSession(category=category)

Türkçe, anadiliniz olduğundan, dilimizi zengin ve yaratıcı biçimlerde kullanmalısın. Lütfen cümlelerinizde benzer yapılar kullanmaktan kaçının.İnsan isimlerini kullanarak cümle kurmamalısın ! İstemde belirtilen kelimeleri gerçek, somut veya fiziki anlama gelecek bir şekilde kullanmalısın.İstemde belirtilen kelimeleri mecaz anlamda kullanmamalısın !


In [733]:
type = "Enhanced-prompting"
step = 3

In [734]:
prompt = f"""
"{word1}" ve "{word2}" bu iki kelimenin fiziksel bir eylemi ifade edecek şekilde kullanılarak geçtiği farklı cümle yapılarıyla 5 cümle oluştur. İstemde belirtilen kelimeleri cümlede istediğin yerleri koyabilirsin ama iki kelimenin de geçtiği cümle kurmalısın. Lütfen cümlelerinizi tekrar kullanmaktan kaçının. Yanıt olarak sadece cümleleri dön.
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


"Ortaya" ve "çıkmak" bu iki kelimenin fiziksel bir eylemi ifade edecek şekilde kullanılarak geçtiği farklı cümle yapılarıyla 5 cümle oluştur. İstemde belirtilen kelimeleri cümlede istediğin yerleri koyabilirsin ama iki kelimenin de geçtiği cümle kurmalısın. Lütfen cümlelerinizi tekrar kullanmaktan kaçının. Yanıt olarak sadece cümleleri dön.





In [735]:
responses

['1. Tatlı bir meltem esintisiyle birlikte bahçenin ortasından çıkmak gerçekten ferahlatıcıydı.\n2. Kutunun içinden ortaya çıkmak için kedi tüm çevikliğini kullandı.\n3. Çadırdan dışarı ilk ortaya çıkan kişi, sabahın erken saatlerinde uyanan arkadaşımız oldu.\n4. Filmlerdeki gibi sisler arasından ortaya çıkmak, o anki sisli havada büyüleyici bir sahne oluşturdu.\n5. Derin bir nefes alarak karanlık koridordan ortaya çıkmak, onun için büyük bir cesaret gerektiriyordu.',
 '1. Nehir kenarında gizlenmiş balıkçıl, birden bire ortaya çıkarak uçmaya başladı.\n2. Perdenin arkasından korkarak ortaya çıkmak, küçük kızın yüzündeki heyecanı belli ediyordu.\n3. Yer altı mağarasından ilk kez ortaya çıkan keşif ekibi, ışığa doğru adımlarını hızlandırdı.\n4. Piknik yaptığımız alanın yanından, bir anda bir ceylan sürüsü ortaya çıkarak etrafımda koşturmaya başladı.\n5. Gösterinin sonunda, sihirbaz birden bire sahnenin tam ortasından ortaya çıktı ve seyircileri şaşkınlığa uğrattı.',
 '1. Köpeğimiz, bahçe 

In [736]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,Tatlı bir meltem esintisiyle birlikte bahçenin...,gerçek,Ortaya çıkmak,Enhanced-prompting
1,Kutunun içinden ortaya çıkmak için kedi tüm çe...,gerçek,Ortaya çıkmak,Enhanced-prompting
2,"Çadırdan dışarı ilk ortaya çıkan kişi, sabahın...",gerçek,Ortaya çıkmak,Enhanced-prompting
3,Filmlerdeki gibi sisler arasından ortaya çıkma...,gerçek,Ortaya çıkmak,Enhanced-prompting
4,Derin bir nefes alarak karanlık koridordan ort...,gerçek,Ortaya çıkmak,Enhanced-prompting
5,"Nehir kenarında gizlenmiş balıkçıl, birden bir...",gerçek,Ortaya çıkmak,Enhanced-prompting
6,"Perdenin arkasından korkarak ortaya çıkmak, kü...",gerçek,Ortaya çıkmak,Enhanced-prompting
7,Yer altı mağarasından ilk kez ortaya çıkan keş...,gerçek,Ortaya çıkmak,Enhanced-prompting
8,"Piknik yaptığımız alanın yanından, bir anda bi...",gerçek,Ortaya çıkmak,Enhanced-prompting
9,"Gösterinin sonunda, sihirbaz birden bire sahne...",gerçek,Ortaya çıkmak,Enhanced-prompting


In [737]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,Sabahleyin bahçeden çıkmak için kapıyı araladı...,gerçek,Ortaya çıkmak,Zero-shot
1,Kutuyu açtığında içinden eski bir saat çıkarak...,gerçek,Ortaya çıkmak,Zero-shot
2,Çadırdan tehlikeli bir ses geldiğini duyunca i...,gerçek,Ortaya çıkmak,Zero-shot
3,"Köpek, koltuğun altına gizlenmiş topunu ortaya...",gerçek,Ortaya çıkmak,Zero-shot
4,"Çocuklar sandığı köşeden çıkarıp açınca, içind...",gerçek,Ortaya çıkmak,Zero-shot
...,...,...,...,...
95,"Köpeğimiz, bahçe kapısının ardından merakla or...",gerçek,Ortaya çıkmak,Enhanced-prompting
96,"Patika yolu takip ederken, ani bir sesle çalıl...",gerçek,Ortaya çıkmak,Enhanced-prompting
97,Karanlık tünelin sonunda ışığı gördüğümüzde hı...,gerçek,Ortaya çıkmak,Enhanced-prompting
98,Kutunun içinden küçük bir yavru kedi çekingen ...,gerçek,Ortaya çıkmak,Enhanced-prompting


In [738]:
df_structured

,submission,category,idiom,type
0,Sabahleyin bahçeden çıkmak için kapıyı araladı...,gerçek,Ortaya çıkmak,Zero-shot
1,Kutuyu açtığında içinden eski bir saat çıkarak...,gerçek,Ortaya çıkmak,Zero-shot
2,Çadırdan tehlikeli bir ses geldiğini duyunca i...,gerçek,Ortaya çıkmak,Zero-shot
3,"Köpek, koltuğun altına gizlenmiş topunu ortaya...",gerçek,Ortaya çıkmak,Zero-shot
4,"Çocuklar sandığı köşeden çıkarıp açınca, içind...",gerçek,Ortaya çıkmak,Zero-shot
...,...,...,...,...
95,"Köpeğimiz, bahçe kapısının ardından merakla or...",gerçek,Ortaya çıkmak,Enhanced-prompting
96,"Patika yolu takip ederken, ani bir sesle çalıl...",gerçek,Ortaya çıkmak,Enhanced-prompting
97,Karanlık tünelin sonunda ışığı gördüğümüzde hı...,gerçek,Ortaya çıkmak,Enhanced-prompting
98,Kutunun içinden küçük bir yavru kedi çekingen ...,gerçek,Ortaya çıkmak,Enhanced-prompting


In [784]:
# Define the path for the CSV file
file_path = 'gpt4_tr_gerçek.csv'

# Check if the file exists
if os.path.exists(file_path):
    # Append without writing headers
    df_structured.to_csv(file_path, mode='a', index=False, header=False)
else:
    # Create a new file and write the DataFrame with headers
    df_structured.to_csv(file_path, mode='w', index=False, header=True)